In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from funkwpap import *
import sympy as sp, pandas as pd, numpy as np, tqdm, time, sys, matplotlib.pyplot as plt, tqdm
from statistics import mean
from scipy.optimize import fsolve
import gurobipy as gb


In [2]:
import sympy as sp

# Define symbolic variable
x = sp.symbols('x')

# Create a Regulator with an emission cap
RegulatorETS = Regulator("ETS", permit_price=20.0, emission_cap=1000000)

# Define Countries with updated names and attributes
country1 = Country('Atlantis', size=50, regulator=RegulatorETS, GDP_per_capita=40000, population=100000000, industry_percentage=30)
country2 = Country('Omashu', size=20, regulator=RegulatorETS, GDP_per_capita=35000, population=50000000, industry_percentage=25)
country3 = Country('Hogsmeade', size=10, regulator=RegulatorETS, GDP_per_capita=45000, population=20000000, industry_percentage=20)
country4 = Country('The_Court_of_Miracles', size=5, regulator=RegulatorETS, GDP_per_capita=30000, population=10000000, industry_percentage=15)
country5 = Country('Lilipoupoli', size=2, regulator=RegulatorETS, GDP_per_capita=25000, population=5000000, industry_percentage=10)

# Define Sectors with demand functions
sector1 = Sector('Steel', price_demand_function=200 - 0.1*x, free_emission_multiplier=0, regulator=RegulatorETS)
sector2 = Sector('Cement', price_demand_function=150 - 0.05*x, free_emission_multiplier=0, regulator=RegulatorETS)
sector3 = Sector('Paper', price_demand_function=100 - 0.02*x, free_emission_multiplier=0, regulator=RegulatorETS)
sector4 = Sector('Chemicals', price_demand_function=250 - 0.15*x, free_emission_multiplier=0, regulator=RegulatorETS)
sector5 = Sector('Automotive', price_demand_function=300 - 0.2*x, free_emission_multiplier=0, regulator=RegulatorETS)
sector6 = Sector('Textiles', price_demand_function=80 - 0.01*x, free_emission_multiplier=0, regulator=RegulatorETS)

# Create Firms with updated definitions (production_cost_function=0, abatement_cost_function as previous production_cost_function)
# Firms in Sector 1: Steel
firm_S1_F1 = Firm('S1_F1', sector1, country1, x*0, 2*x + 3*x**2 + x**3, 10000, 5000, 0, RegulatorETS)
firm_S1_F2 = Firm('S1_F2', sector1, country1, x*0, 3*x + 2*x**2 + 2*x**3, 12000, 6000, 0, RegulatorETS)
firm_S1_F3 = Firm('S1_F3', sector1, country2, x*0, 4*x + x**2 + 3*x**3, 8000, 4000, 0, RegulatorETS)
firm_S1_F4 = Firm('S1_F4', sector1, country3, x*0, 2*x + 2*x**2 + 2*x**3, 7000, 3500, 0, RegulatorETS)
firm_S1_F5 = Firm('S1_F5', sector1, country4, x*0, 3*x + 3*x**2 + x**3, 6000, 3000, 0, RegulatorETS)

# Firms in Sector 2: Cement
firm_S2_F1 = Firm('S2_F1', sector2, country1, x*0, x + 2*x**2 + 3*x**3, 9000, 4500, 0, RegulatorETS)
firm_S2_F2 = Firm('S2_F2', sector2, country1, x*0, 2*x + x**2 + 2*x**3, 9500, 4750, 0, RegulatorETS)
firm_S2_F3 = Firm('S2_F3', sector2, country2, x*0, 3*x + x**2 + x**3, 7000, 3500, 0, RegulatorETS)
firm_S2_F4 = Firm('S2_F4', sector2, country3, x*0, x + 3*x**2 + x**3, 6000, 3000, 0, RegulatorETS)
firm_S2_F5 = Firm('S2_F5', sector2, country4, x*0, 2*x + 2*x**2 + x**3, 5000, 2500, 0, RegulatorETS)

# Firms in Sector 3: Paper
firm_S3_F1 = Firm('S3_F1', sector3, country3, x*0, x + x**2 + x**3, 5000, 10000, 0, RegulatorETS)
firm_S3_F2 = Firm('S3_F2', sector3, country4, x*0, x + 2*x**2 + x**3, 4500, 9000, 0, RegulatorETS)
firm_S3_F3 = Firm('S3_F3', sector3, country5, x*0, x + x**2 + 2*x**3, 3000, 6000, 0, RegulatorETS)
firm_S3_F4 = Firm('S3_F4', sector3, country2, x*0, 2*x + x**2 + x**3, 5500, 11000, 0, RegulatorETS)
firm_S3_F5 = Firm('S3_F5', sector3, country1, x*0, x + 2*x**2 + 2*x**3, 6000, 12000, 0, RegulatorETS)

# Firms in Sector 4: Chemicals
firm_S4_F1 = Firm('S4_F1', sector4, country1, x*0, 3*x + 4*x**2 + x**3, 11000, 5500, 0, RegulatorETS)
firm_S4_F2 = Firm('S4_F2', sector4, country1, x*0, 4*x + 3*x**2 + 2*x**3, 10500, 5250, 0, RegulatorETS)
firm_S4_F3 = Firm('S4_F3', sector4, country2, x*0, 2*x + 5*x**2 + x**3, 9000, 4500, 0, RegulatorETS)
firm_S4_F4 = Firm('S4_F4', sector4, country3, x*0, 3*x + 3*x**2 + 3*x**3, 8000, 4000, 0, RegulatorETS)
firm_S4_F5 = Firm('S4_F5', sector4, country4, x*0, 2*x + 2*x**2 + 4*x**3, 7000, 3500, 0, RegulatorETS)

# Firms in Sector 5: Automotive
firm_S5_F1 = Firm('S5_F1', sector5, country1, x*0, 5*x + 4*x**2 + x**3, 12000, 6000, 0, RegulatorETS)
firm_S5_F2 = Firm('S5_F2', sector5, country2, x*0, 4*x + 5*x**2 + 2*x**3, 10000, 5000, 0, RegulatorETS)
firm_S5_F3 = Firm('S5_F3', sector5, country3, x*0, 3*x + 3*x**2 + 2*x**3, 9000, 4500, 0, RegulatorETS)
firm_S5_F4 = Firm('S5_F4', sector5, country4, x*0, 2*x + 4*x**2 + 3*x**3, 8000, 4000, 0, RegulatorETS)
firm_S5_F5 = Firm('S5_F5', sector5, country5, x*0, x + 3*x**2 + 4*x**3, 7000, 3500, 0, RegulatorETS)

# Firms in Sector 6: Textiles
firm_S6_F1 = Firm('S6_F1', sector6, country5, x*0, x + x**2 + x**3, 2000, 10000, 0, RegulatorETS)
firm_S6_F2 = Firm('S6_F2', sector6, country4, x*0, x + 2*x**2 + x**3, 2500, 12500, 0, RegulatorETS)
firm_S6_F3 = Firm('S6_F3', sector6, country3, x*0, x + x**2 + 2*x**3, 3000, 15000, 0, RegulatorETS)
firm_S6_F4 = Firm('S6_F4', sector6, country2, x*0, 2*x + x**2 + x**3, 3500, 17500, 0, RegulatorETS)
firm_S6_F5 = Firm('S6_F5', sector6, country1, x*0, x + 2*x**2 + 2*x**3, 4000, 20000, 0, RegulatorETS)


In [3]:
# RegulatorETS.BAU_calculator(precision = 0.01, print_diff = True)
# 17638.87773
RegulatorETS.emission_cap = 17638.87773*0.8

In [ ]:
RegulatorETS.optimization_concave_formulation_ab(print_output= True, gurobi_print=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-14
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i9-9900 CPU @ 3.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 91 rows, 61 columns and 330 nonzeros
Model fingerprint: 0x57f510e7
Model has 30 quadratic objective terms
Model has 30 quadratic constraints
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  QMatrix range    [3e+00, 1e+01]
  QLMatrix range   [1e+00, 1e+01]
  Objective range  [6e-02, 1e+01]
  QObjective range [6e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 1e+04]
  QRHS range       [1e+00, 5e+00]
Presolve removed 30 rows and 0 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 30 rows and 0 columns
Presolve time: 0.00s
Presolved: 91 rows, 62 columns, 360 nonzeros
Presolv